In [1]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# data preparation, enter in our csv file, or json if we go that route
real_estate_df = pd.read_csv(Path('/Users/skylarwhite/Desktop/Final Project/Real_Estate_Final_Project/Resources/Clean_Real_Estate.csv'))
real_estate_df.head()

,price,bed,bath,acre_lot,full_address,city,state,zip_code,house_size,TJs_store_count,walmart_store_count
0,180000.0,2.0,1.0,0.34,"23 Moore St, Agawam, MA, 01001",Agawam,Massachusetts,1001,676.0,0.0,0.0
1,169900.0,2.0,2.0,0.00,"420 Main St Apt 42, Agawam, MA, 01001",Agawam,Massachusetts,1001,892.0,0.0,0.0
2,242000.0,2.0,2.0,0.00,"2A Mansion Woods Dr Unit 2A, Agawam, MA, 01001",Agawam,Massachusetts,1001,1428.0,0.0,0.0
3,299950.0,2.0,2.0,0.00,"19 Castle Hill Rd Unit C, Agawam, MA, 01001",Agawam,Massachusetts,1001,1659.0,0.0,0.0
4,239900.0,3.0,1.0,0.46,"270 South St, Agawam, MA, 01001",Agawam,Massachusetts,1001,1196.0,0.0,0.0


In [3]:
#Add Walmart and Trader Joes Data as dataframe
trader_joes_df = pd.read_csv(Path('/Users/skylarwhite/Desktop/Final Project/Real_Estate_Final_Project/Clean_TJs.csv'))
walmart_df = pd.read_csv(Path('/Users/skylarwhite/Desktop/Final Project/Real_Estate_Final_Project/Clean_Walmart.csv'))

In [4]:
real_estate_df.corr()

,price,bed,bath,acre_lot,zip_code,house_size,TJs_store_count,walmart_store_count
price,1.000000,0.033587,0.292184,-0.004027,0.097698,0.112735,0.229356,-0.136975
bed,0.033587,1.000000,0.546418,-0.002606,-0.081938,0.380763,-0.137870,0.057945
bath,0.292184,0.546418,1.000000,-0.002882,-0.037444,0.382849,-0.026944,0.005224
acre_lot,-0.004027,-0.002606,-0.002882,1.000000,0.004532,-0.001289,-0.003194,-0.002390
zip_code,0.097698,-0.081938,-0.037444,0.004532,1.000000,-0.056149,0.099865,-0.066804
house_size,0.112735,0.380763,0.382849,-0.001289,-0.056149,1.000000,-0.043697,0.013071
TJs_store_count,0.229356,-0.137870,-0.026944,-0.003194,0.099865,-0.043697,1.000000,-0.011902
walmart_store_count,-0.136975,0.057945,0.005224,-0.002390,-0.066804,0.013071,-0.011902,1.000000


Begin the machine learning model with Extra Trees Regressor--Using the binary values for the Trader Joes and Walmart as features

In [5]:
#Define the values to be used in the model
real_estate_model=real_estate_df[["price","bed","bath","house_size","TJs_store_count","walmart_store_count"]]

In [6]:
#define X
import numpy as np
X=real_estate_df[["bed","bath","acre_lot","house_size","TJs_store_count","walmart_store_count"]]
X


,bed,bath,acre_lot,house_size,TJs_store_count,walmart_store_count
0,2.0,1.0,0.34,676.0,0.0,0.0
1,2.0,2.0,0.00,892.0,0.0,0.0
2,2.0,2.0,0.00,1428.0,0.0,0.0
3,2.0,2.0,0.00,1659.0,0.0,0.0
4,3.0,1.0,0.46,1196.0,0.0,0.0
...,...,...,...,...,...,...
536925,2.0,2.0,0.00,1248.0,0.0,0.0
536926,5.0,1.0,35.90,1586.0,0.0,0.0
536927,2.0,2.0,0.00,1680.0,0.0,0.0
536928,5.0,2.0,0.00,1846.0,0.0,0.0


In [7]:
#Define y predictor variable
y=real_estate_df['price']

In [8]:
#split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
#define the model
real_estate_model=ExtraTreesRegressor(n_estimators=128, random_state=78)

In [10]:
#Fit the training data to the model
real_estate_model=real_estate_model.fit(X_train, y_train)

In [11]:
#get the accuracy score of the model
real_estate_model.score(X_test, y_test)

0.9110597765243158

In [12]:
import joblib
from joblib import dump,load

joblib.dump(real_estate_model, 'real_estate.joblib')

['real_estate.joblib']

In [13]:
# Calculate predictions

pred = real_estate_model.predict(X_test)
pred

array([ 600943.63380282,  159000.        ,  338000.        , ...,
        554750.89583333, 1595000.        ,  321285.71428571])

In [14]:
print(pred[0])

600943.6338028164


In [15]:
pd.DataFrame({"features":X_train.columns,"importances":real_estate_model.feature_importances_}).sort_values(by="importances",ascending=False)

,features,importances
3,house_size,0.488069
2,acre_lot,0.243449
1,bath,0.128714
4,TJs_store_count,0.063951
0,bed,0.043785
5,walmart_store_count,0.032031
